In [1]:
!nvidia-smi

Fri Jul  9 04:54:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install transformers

     |████████████████████████████████| 2.5MB 8.6MB/s 
     |████████████████████████████████| 3.3MB 57.3MB/s 
     |████████████████████████████████| 901kB 38.5MB/s 


In [4]:
import os
import matplotlib as plt
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import re,nltk,json, pickle
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM,GRU,Bidirectional
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_auc_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import re
from re import sub
import multiprocessing
import os
from time import time 
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Activation,Embedding,Flatten,Bidirectional,MaxPooling1D, Conv1D, MaxPooling1D, GlobalMaxPool1D
from keras.optimizers import SGD,Adam
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import h5py
import csv
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification,XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from transformers import EarlyStoppingCallback


In [5]:
file  = '/content/drive/MyDrive/stop_news.csv'
dataset=pd.read_csv(file,error_bad_lines=False, engine="python" ) # engine ="c" 
print(f"There are {len(dataset)} articles in the dataset.")

There are 28800 articles in the dataset.


In [6]:
from sklearn.preprocessing import LabelEncoder

catlabel = LabelEncoder()
label = catlabel.fit_transform(dataset['category'])


print(set(label))
print(catlabel.classes_)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
['accident' 'art' 'crime' 'economics' 'education' 'entertainment'
 'environment' 'international' 'opinion' 'politics' 'science_tech'
 'sports']


In [7]:
news_text = dataset['news']
X,X_test,y,y_test = train_test_split(news_text,label,train_size = 0.8,
                                                  test_size = 0.2,random_state =42)
X_train,X_valid,y_train,y_valid = train_test_split(X,y,train_size = 0.8,
                                                  test_size = 0.2,random_state =42)

In [8]:
model_name = "neuralspace-reverie/indic-transformers-bn-bert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=12)

Some weights of the model checkpoint at neuralspace-reverie/indic-transformers-bn-bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

In [9]:
X_train = list(X_train)
y_train = list(y_train)

X_test = list(X_test)
y_test = list(y_test)

X_valid = list(X_valid)
y_valid = list(y_valid)

X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=350)
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=350)
X_val_tokenized = tokenizer(X_valid, padding=True, truncation=True, max_length=350)

In [10]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_valid)

In [11]:
# Define Trainer parameters'''
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average="macro")
    precision = precision_score(y_true=labels, y_pred=pred, average="macro")
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# Define Trainer
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/8020w0016e5ep3/",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    learning_rate=6e-5,
    weight_decay=0.001,
    save_steps=3000,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

# Train pre-trained model


In [12]:
trainer.train()

***** Running training *****
  Num examples = 18432
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1728


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
500,0.401600,0.242275,0.927951,0.928908,0.927604,0.927460
1000,0.144600,0.238029,0.937283,0.938426,0.936592,0.936711
1500,0.068100,0.231210,0.946181,0.946174,0.945998,0.945977


***** Running Evaluation *****
  Num examples = 4608
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/8020w0016e5ep3/checkpoint-500
Configuration saved in /content/drive/MyDrive/8020w0016e5ep3/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/8020w0016e5ep3/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4608
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/8020w0016e5ep3/checkpoint-1000
Configuration saved in /content/drive/MyDrive/8020w0016e5ep3/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/8020w0016e5ep3/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4608
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/8020w0016e5ep3/checkpoint-1500
Configuration saved in /content/drive/MyDrive/8020w0016e5ep3/checkpoint-1500/config.json
Model weights saved in /content/drive/MyDrive/8020w0016e5ep3/checkpoint-1500/pytorch_model

TrainOutput(global_step=1728, training_loss=0.1833559865200961, metrics={'train_runtime': 2715.4613, 'train_samples_per_second': 20.363, 'train_steps_per_second': 0.636, 'total_flos': 1.56181596512256e+16, 'train_loss': 0.1833559865200961, 'epoch': 3.0})

In [13]:
# ----- Predict -----#
test_dataset = Dataset(X_test_tokenized)

# Load trained model
model_path = "/content/drive/MyDrive/8020w0016e5ep3/checkpoint-1500"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=12)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

loading configuration file /content/drive/MyDrive/8020w0016e5ep3/checkpoint-1500/config.json
Model config BertConfig {
  "_name_or_path": "neuralspace-reverie/indic-transformers-bn-bert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-12,
  "max_posi

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       464
           1       0.90      0.92      0.91       488
           2       0.92      0.91      0.92       493
           3       0.96      0.94      0.95       459
           4       0.99      0.99      0.99       470
           5       0.97      0.96      0.96       520
           6       0.95      0.93      0.94       465
           7       0.95      0.97      0.96       474
           8       0.92      0.92      0.92       464
           9       0.94      0.95      0.95       478
          10       0.95      0.97      0.96       481
          11       0.97      0.97      0.97       504

    accuracy                           0.95      5760
   macro avg       0.95      0.95      0.95      5760
weighted avg       0.95      0.95      0.95      5760



In [15]:
print(confusion_matrix(y_test, y_pred))
print('\nbert-Accuracy: {0:.2f}%'.format(accuracy_score(y_test, y_pred)*100))
print('\n ep3b32_80_20w0')

[[442   1  12   0   0   0   5   3   0   0   0   1]
 [  1 447   0   0   1   7   1   1  17   0   4   9]
 [ 11   1 450   4   0   0   3   0   1  23   0   0]
 [  2   0   3 430   0   2   4   4   4   1   7   2]
 [  0   1   2   0 463   1   0   0   1   0   2   0]
 [  0   8   2   2   0 497   0   2   3   1   5   0]
 [  4  11   5   5   0   0 433   1   3   2   1   0]
 [  0   1   1   3   0   2   1 458   3   2   3   0]
 [  0  17   0   2   1   3   3   4 429   2   1   2]
 [  0   0  13   1   2   0   2   1   3 456   0   0]
 [  0   3   0   2   2   1   2   4   1   0 465   1]
 [  2   8   0   0   0   1   2   2   1   0   1 487]]

bert-Accuracy: 94.74%

 ep3b32_80_20w0
